In [1]:
import os
os.environ['DJ_HOST']="db.datajoint.com"
os.environ['DJ_USER']="microns"
os.environ['DJ_PASS']="microns2023"
os.environ['DATABASE_PREFIX']="microns_"

import datajoint as dj
from microns_phase3 import nda, utils
 

Connecting microns@db.datajoint.com:3306


/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np

In [3]:
scan_df = pd.DataFrame(nda.Scan & nda.ScanInclude)

In [4]:
%%time 

for _, scan_row in scan_df.iterrows():
    scan_key = scan_row[["session", "scan_idx"]].astype(int).to_dict()
    
    out_path = f"../scratch/traces_session-{scan_key['session']}_scan-{scan_key['scan_idx']}"
    if os.path.exists(out_path): continue
    
    field_df = pd.DataFrame(nda.Field & scan_key)

    trace_dfs = []
    for _, field_row in field_df.iterrows():
        field_key = field_row[["session", "scan_idx", "field"]].astype(int).to_dict()
        %time trace_dfs.append(pd.DataFrame((nda.Fluorescence() & field_key).fetch()))
        
    pd.concat(trace_dfs).reset_index(drop=True).to_feather(out_path, compression="zstd")
    print(f"written: {out_path}")


CPU times: user 784 ms, sys: 160 ms, total: 944 ms
Wall time: 2.8 s
CPU times: user 517 ms, sys: 52.4 ms, total: 570 ms
Wall time: 1.71 s
CPU times: user 1.62 s, sys: 300 ms, total: 1.92 s
Wall time: 7.04 s
CPU times: user 1.71 s, sys: 233 ms, total: 1.94 s
Wall time: 7.63 s
CPU times: user 1.6 s, sys: 279 ms, total: 1.88 s
Wall time: 5.09 s
CPU times: user 1.55 s, sys: 295 ms, total: 1.85 s
Wall time: 4.98 s
CPU times: user 997 ms, sys: 136 ms, total: 1.13 s
Wall time: 3.17 s
CPU times: user 813 ms, sys: 128 ms, total: 942 ms
Wall time: 2.64 s
written: ../scratch/traces_session-4_scan-7
CPU times: user 1.62 s, sys: 233 ms, total: 1.86 s
Wall time: 4.78 s
CPU times: user 1.47 s, sys: 144 ms, total: 1.62 s
Wall time: 4.6 s
CPU times: user 1.61 s, sys: 167 ms, total: 1.77 s
Wall time: 5.58 s
CPU times: user 1.81 s, sys: 174 ms, total: 1.98 s
Wall time: 8.7 s
CPU times: user 1.18 s, sys: 184 ms, total: 1.37 s
Wall time: 6.68 s
CPU times: user 1.21 s, sys: 139 ms, total: 1.35 s
Wall time: 

In [5]:
%%time 

mean_intesnsity_df = pd.DataFrame(nda.MeanIntensity())

CPU times: user 397 ms, sys: 37.9 ms, total: 434 ms
Wall time: 9.35 s


In [9]:
f"../scratch/mean_intensities"

mean_intesnsity_df.to_feather(out_path, compression="zstd")